In [ ]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
# Log in to wandb. Replace "your_api_key_here" with your actual WANDB API key.
wandb.login(key="your_api_key_here")

from transformers import (AutoTokenizer, AutoModelForSequenceClassification, Trainer,
                          TrainingArguments, EarlyStoppingCallback)
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 23.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Load and preprocess the new dataset (Movies_and_TV)
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Movies_and_TV", trust_remote_code=True)

# Shuffle the dataset and select a subset
shuffled_dataset = dataset["full"].shuffle(seed=42)
subset_size = int(0.2 * len(shuffled_dataset))  # use 20% of the data
subset_dataset = shuffled_dataset.select(range(subset_size))

# Filter out samples where the rating equals 3
subset_dataset = subset_dataset.filter(lambda x: x["rating"] != 3)

# Initialize the DeBERTa tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-small")

# Define the tokenization and formatting function
def tokenize_and_format(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Tokenize the new dataset
tokenized_dataset = subset_dataset.map(tokenize_and_format, batched=True)

# Save a checkpoint of the tokenized dataset
tokenized_dataset.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_movies_checkpoint")

# Split into train/test and set to PyTorch tensors
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


In [ ]:
# Step 2': Load from checkpoint if needed
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_movies_checkpoint")
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


In [ ]:
# Step 3: Load the previously fine-tuned DeBERTa model from the CDs training
model = AutoModelForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_CDs_deberta", num_labels=2
)

# Define the compute_metrics function for evaluation
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
# Step 4: Set up TrainingArguments for continued fine-tuning
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FP/Movies_results_continued",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=6,
    warmup_steps=500,
    report_to=["wandb"],
    run_name="deberta_movies_and_tv_finetune",
    logging_steps=50,
    logging_first_step=True
)

# Print number of training and evaluation samples
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])
print("Total training samples:", total_train)
print("Total evaluation samples:", total_eval)

# Use partial data for actual training and eval
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.05 * total_train)))
eval_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.05 * total_eval)))

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Total training samples: 2571408
Total evaluation samples: 642852


In [ ]:
# Step 5: Continue fine-tuning
train_result = trainer.train()

# Print checkpoint and metrics
print("Best model checkpoint:", trainer.state.best_model_checkpoint)
print("Best validation metric:", trainer.state.best_metric)

eval_results = trainer.evaluate()
print("Evaluation results of the best model:", eval_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.095100,0.084693,0.968981,0.968496,0.968397,0.968981
2,0.065600,0.078376,0.972279,0.971973,0.971863,0.972279
3,0.044100,0.089033,0.972435,0.972432,0.972430,0.972435
4,0.032300,0.101287,0.971657,0.971783,0.971943,0.971657


Could not locate the best model at /content/drive/MyDrive/FP/Movies_results_continued/checkpoint-2010/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


Best model checkpoint: /content/drive/MyDrive/FP/Movies_results_continued/checkpoint-2010
Best validation metric: 0.07837598025798798


Evaluation results of the best model: {'eval_loss': 0.10128685086965561, 'eval_accuracy': 0.9716570219650302, 'eval_f1': 0.9717833744396753, 'eval_precision': 0.9719426493322775, 'eval_recall': 0.9716570219650302, 'eval_runtime': 76.6332, 'eval_samples_per_second': 419.426, 'eval_steps_per_second': 6.564, 'epoch': 4.0}


In [ ]:
# Save the final model
trainer.save_model("/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_movies_deberta")